In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib as jb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
df = jb.load('../data/generated/df_newimages_bands_downside.pkl.z')
df.head()

In [ ]:
features = df.drop(['x', 'y', 'z'], axis=1)
target = df.z

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(features, target, random_state=42, test_size=0.3)

In [ ]:
def build_model(train_dataset):
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
      ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

In [ ]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

In [ ]:
EPOCHS = 1000
model = build_model(X_train)

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.2, verbose=0, callbacks=[PrintDot()])

In [58]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.2, verbose=0, callbacks=[early_stop, PrintDot()])


....................

In [63]:
p = model.predict(X_val).flatten()

In [65]:
from sklearn.metrics import r2_score

In [66]:
r2_score(y_val, p)

0.5986274465997139

In [88]:
th = 1500
X_train = df[df.x <= th].drop(['x', 'y', 'z'], axis=1)
y_train = df[df.x <= th].z


X_val = df[df.x > th].drop(['x', 'y', 'z'], axis=1)
y_val = df[df.x > th].z

In [89]:
len(X_train), len(X_val)

(14514, 5183)

In [90]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.2, verbose=0, callbacks=[early_stop, PrintDot()])


............

In [91]:
p = model.predict(X_val).flatten()

In [92]:
r2_score(y_val, p)

0.007928124516909496